## Detection of Deep Network Generated Images Using Disparities in Color Components
Amar kumar (Ced17i029)
 
Saumya Prakash (Ced17i043)

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import os
import math


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Important functions


In [0]:
filter1 = np.array([1,-1])
filter2 = np.array([[1],[-1]])
# rotates a matrix by 180 degree
########################################
def rotateby180(arr):
    arr = np.transpose(arr)
    for i,ival in enumerate(arr):
        arr[i] = np.flipud(arr[i])
    arr = np.transpose(arr)
    return arr
#########################################


# High pass filter(1 x n)
def HPF_row(array_of_colors,filters):
  l1 = []
  for row in array_of_colors:
    a = np.convolve(row,filters,'same')
    l1.append(a)
  return l1


# High pass filter(n x 1)
########################################################
def HPF_column(arr,filt):
#     appending a row of 0
    arr = np.vstack([arr, [0]*arr.shape[1]])
    arr = rotateby180(arr)
    l = []
    for i in range(0,arr.shape[1]):
        temp = []
        for j in range(0,arr.shape[0]-1):
            temp.append(arr[j][i]*filt[0] + arr[j+1][i]*filt[1])
        l.append(temp)
    m = np.transpose(l)[0]
    m = rotateby180(m)
    return m
####################################################


# Binarization of RGB residuals
def residualToBin(residual):
  li = []
  for i in residual:
    temp = []
    for j in i:
      if j >0:
        temp.append(1)
      else:
        temp.append(0)
    li.append(temp)
  return np.array(li)

# Truncation of chrominance residuals

def residualToTrunc(residual,t):
  li = []
  for i in residual:
    temp = []
    for j in i:
      if j >=t:
        temp.append(t)
      elif j<= -t:
        temp.append(-t)
      else:
        temp.append(j)
    li.append(temp)
  return np.array(li)
  
# co occurence matrix

def uniqCount1(a,b,matrix):
  count = 0
  for i,ival in enumerate(matrix):
    for j,jval in enumerate(ival):
      if j== np.shape(matrix)[1]-1:
        break
      if jval==a and matrix[i][j+1]==b:
        count+=1
  return count

def getCoOccurenceMatrix1(matrix):
  uniq = np.unique(matrix)
  coMatrix=[]
  for i in uniq:
    temp = []
    for j in uniq:
      count = uniqCount1(i,j,matrix)
      temp.append(count)
    coMatrix.append(temp)
  return np.array(coMatrix)

#################################################################
def uniqCount2(a,b,matrix):
  count = 0
  for i,ival in enumerate(matrix):
    for j,jval in enumerate(ival):
      if i== np.shape(matrix)[0]-1:
        break
      if jval==a and matrix[i+1][j]==b:
        count+=1
  return count


def getCoOccurenceMatrix2(matrix):
  uniq = np.unique(matrix)
  coMatrix=[]
  for i in uniq:
    temp = []
    for j in uniq:
      count = uniqCount2(i,j,matrix)
      temp.append(count)
    coMatrix.append(temp)
  return np.array(coMatrix)
################################################################

# Feature Extractor function

In [0]:
def feature_extractor(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  imgHSV = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
  imgYCbCr = cv2.cvtColor(img,cv2.COLOR_RGB2YCR_CB)
  iR = np.array(img[:,:,0])
  iG = np.array(img[:,:,1])
  iB = np.array(img[:,:,2])
  iH = np.array(imgHSV[:,:,0])
  iS = np.array(imgHSV[:,:,1])
  iCr = np.array(imgYCbCr[:,:,1])
  iCb = np.array(imgYCbCr[:,:,2])
  rR1 = HPF_row(iR,filter1)
  rB1 = HPF_row(iB,filter1)
  rG1 = HPF_row(iG,filter1)
  rH1 = HPF_row(iH,filter1)
  rS1 = HPF_row(iS,filter1)
  rCr1 = HPF_row(iCr,filter1)
  rCb1 = HPF_row(iCb,filter1)
  ###########################
  rR2 = HPF_column(iR,filter2)
  rB2 = HPF_column(iB,filter2)
  rG2 = HPF_column(iG,filter2)
  rH2 = HPF_column(iH,filter2)
  rS2 = HPF_column(iS,filter2)
  rCr2 = HPF_column(iCr,filter2)
  rCb2 = HPF_column(iCb,filter2)


  #binarization
  bR1 = residualToBin(rR1)
  bB1 = residualToBin(rB1)
  bG1 = residualToBin(rG1)
  #######################
  bR2 = residualToBin(rR2)
  bB2 = residualToBin(rB2)
  bG2 = residualToBin(rG2)

  # assembled residual image R̂ RGB
  # R̂ RGB = R̂ R · 2^0 + R̂ G · 2^1 + R̂ B · 2^2
  R_RGB1 = bR1 + bG1*2 + bB1*4
  #############################
  R_RGB2 = bR2 + bG2*2 + bB2*4
  # shape of R_RGB is 128 x 128
  # Truncation of chrominance residuals
  t = 2 # truncated thresh.
  tH1 = residualToTrunc(rH1,t)
  tS1 = residualToTrunc(rS1,t)
  tCb1 = residualToTrunc(rCb1,t)
  tCr1 = residualToTrunc(rCr1,t)
  ##############################
  tH2 = residualToTrunc(rH2,t)
  tS2 = residualToTrunc(rS2,t)
  tCb2 = residualToTrunc(rCb2,t)
  tCr2 = residualToTrunc(rCr2,t)

  coH1 = getCoOccurenceMatrix1(tH1)
  # coH1 = coH1/np.sum(coH1)
  coS1 = getCoOccurenceMatrix1(tS1)
  # coS1 = coS1/np.sum(coS1)
  coCb1 = getCoOccurenceMatrix1(tCb1)
  # cocb1 = coCb1/np.sum(coCb1)
  coCr1 = getCoOccurenceMatrix1(tCr1)
  # coCr1 = coCr1/np.sum(coCr1)
  coRGB1 = getCoOccurenceMatrix1(R_RGB1)
  # coRGB1 = coRGB1/np.sum(coRGB1)
  ##############################
  coH2 = getCoOccurenceMatrix1(tH2)
  # coH2 = coH2/np.sum(coH2)
  coS2 = getCoOccurenceMatrix1(tS2)
  # coS2 = coS2/np.sum(coS2)
  coCb2 = getCoOccurenceMatrix1(tCb2)
  # cocb2 = coCb2/np.sum(coCb2)
  coCr2 = getCoOccurenceMatrix1(tCr2)
  # coCr2 = coCr2/np.sum(coCr2)
  coRGB2 = getCoOccurenceMatrix1(R_RGB2)
  # coRGB2 = coRGB2/np.sum(coRGB2)
  ####################################
  coH3 = getCoOccurenceMatrix2(tH1)
  # coH3 = coH3/np.sum(coH3)
  coS3 = getCoOccurenceMatrix2(tS1)
  # coS3 = coS3/np.sum(coS3)
  coCb3 = getCoOccurenceMatrix2(tCb1)
  # cocb3 = coCb3/np.sum(coCb3)
  coCr3 = getCoOccurenceMatrix2(tCr1)
  # coCr3 = coCr3/np.sum(coCr3)
  coRGB3 = getCoOccurenceMatrix2(R_RGB1)
  # coRGB3 = coRGB3/np.sum(coRGB3)
  ##############################
  coH4 = getCoOccurenceMatrix2(tH2)
  # coH4 = coH4/np.sum(coH4)
  coS4 = getCoOccurenceMatrix2(tS2)
  # coS4 = coS4/np.sum(coS4)
  coCb4 = getCoOccurenceMatrix2(tCb2)
  # cocb4 = coCb4/np.sum(coCb4)
  coCr4 = getCoOccurenceMatrix2(tCr2)
  # coCr4 = coCr4/np.sum(coCr4)
  coRGB4 = getCoOccurenceMatrix2(R_RGB2)
  # coRGB4 = coRGB4/np.sum(coRGB4)

  coH = np.mean([coH1,coH2,coH3,coH4], axis = 0)
  coH_T = np.transpose(coH)
  coH = coH + coH_T # making co-occurence matrix symmetric
  coH = coH/np.sum(coH) # normalization
  ###############################################
  coS = np.mean([coS1,coS2,coS3,coS4], axis = 0)
  coS_T = np.transpose(coS)
  coS = coS + coS_T # making co-occurence matrix symmetric
  coS = coS/np.sum(coS) # normalization
  #################################################
  coCb = np.mean([coCb1,coCb2,coCb3,coCb4], axis = 0)
  coCb_T = np.transpose(coCb)
  coCb = coCb + coCb_T # making co-occurence matrix symmetric
  coCb = coCb/np.sum(coCb) # normalization
  ################################################
  coCr = np.mean([coCr1,coCr2,coCr3,coCr4], axis = 0)
  coCr_T = np.transpose(coCr)
  coCr = coCr + coCr_T # making co-occurence matrix symmetric
  coCr = coCr/np.sum(coCr) # normalization
  ###################################################
  coRGB = np.mean([coRGB1,coRGB2,coRGB3,coRGB4], axis = 0)
  coRGB_T = np.transpose(coRGB)
  coRGB = coRGB + coRGB_T # making co-occurence matrix symmetric
  coRGB = coRGB/np.sum(coRGB) # normalization
  feature = np.concatenate([coRGB.flatten(),coH.flatten(),coS.flatten(),coCb.flatten(),coCr.flatten()])
  
  return feature

# Generating 500 Real image and 500 Fake image feature vector and saving to drive


In [0]:
# Generating 500 real image feature vector
data = []
path = "/content/drive/My Drive/Colab Notebooks/dataset/cropped/img_align_celeba"
dirs = os.listdir(path)
print(len(dirs))
count=0
for item in dirs:
  count+=1
  if(count==501):
    break
  print(count)
  fullpath = os.path.join(path,item)         #corrected
  # print(fullpath)
  if os.path.isfile(fullpath):
    imgtemp = cv2.imread('/content/drive/My Drive/Colab Notebooks/dataset/cropped/img_align_celeba/'+str(item))
    # print("start")
    data.append(feature_extractor(imgtemp))
    # print("end")

df = pd.DataFrame(data)
y=[]
for i in data:
  y.append(0)
df['y']=y
df.to_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresReal_amarCo.csv')

# Generating 500 fake image feature vector

data2 = []
path = "/content/drive/My Drive/Colab Notebooks/dataset/dataset_celebA"
dirs = os.listdir(path)
print(len(dirs))
count=0
for item in dirs:
  count+=1
  if(count==501):
    break
  print(count)
  fullpath = os.path.join(path,item)         #corrected
  # print(fullpath)
  if os.path.isfile(fullpath):
    imgtemp = cv2.imread('/content/drive/My Drive/Colab Notebooks/dataset/dataset_celebA/'+str(item))
    # print("start")
    data2.append(feature_extractor(imgtemp))
    # print("end")
df2 = pd.DataFrame(data2)
y=[]
for i in data2:
  y.append(1)
df2['y']=y
df2.to_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresFake_amarCo.csv')
df2

# Importing generated feature vector from drive

In [5]:
# loading saved features 
real = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresReal_amarCo.csv')
fake = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresFake_amarCo.csv')
df = real
df2 = fake
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,y
0,0,0.334523,0.012219,0.006997,0.007720,0.011519,0.004744,0.007128,0.097272,0.012219,0.005383,0.000607,0.001553,0.000454,0.001130,0.000154,0.007636,0.006997,0.000607,0.001784,0.000707,0.000423,0.000384,0.000830,0.003360,0.007720,0.001553,0.000707,0.003368,0.000169,0.000469,0.000208,0.010227,0.011519,0.000454,0.000423,0.000169,0.005244,0.001038,0.001292,...,0.078302,0.394962,0.063315,0.004052,0.000830,0.007151,0.063315,0.074726,0.011596,0.000938,0.001084,0.004052,0.011596,0.041277,0.044061,0.014218,0.003491,0.000546,0.000108,0.014218,0.077710,0.058724,0.006682,0.000592,0.003491,0.058724,0.382720,0.073934,0.004321,0.000546,0.006682,0.073934,0.104761,0.015348,0.000108,0.000592,0.004321,0.015348,0.034818,0
1,1,0.323188,0.014356,0.005644,0.007220,0.008681,0.004914,0.005890,0.117018,0.014356,0.006828,0.000254,0.002953,0.000254,0.000938,0.000054,0.006944,0.005644,0.000254,0.001492,0.000415,0.000215,0.000069,0.000546,0.004621,0.007220,0.002953,0.000415,0.003137,0.000062,0.000200,0.000169,0.007367,0.008681,0.000254,0.000215,0.000062,0.003245,0.000607,0.001461,...,0.079017,0.311193,0.072396,0.003260,0.000254,0.006221,0.072396,0.105792,0.013080,0.000377,0.000500,0.003260,0.013080,0.028082,0.035679,0.013587,0.003768,0.000577,0.000277,0.013587,0.098256,0.071897,0.006436,0.000454,0.003768,0.071897,0.302288,0.076456,0.004460,0.000577,0.006436,0.076456,0.104223,0.018547,0.000277,0.000454,0.004460,0.018547,0.066637,0
2,2,0.267901,0.021707,0.010504,0.007836,0.016994,0.005675,0.014072,0.075910,0.021707,0.023314,0.001053,0.007144,0.000638,0.002491,0.000331,0.012511,0.010504,0.001053,0.005675,0.000923,0.001407,0.000254,0.002461,0.004375,0.007836,0.007144,0.000923,0.008828,0.000092,0.000692,0.000292,0.012126,0.016994,0.000638,0.001407,0.000092,0.011580,0.001000,0.006390,...,0.069013,0.282742,0.064922,0.004906,0.000838,0.007436,0.064922,0.092996,0.019700,0.000638,0.000800,0.004906,0.019700,0.079355,0.115727,0.024007,0.006997,0.001269,0.000769,0.024007,0.088583,0.059463,0.007497,0.001584,0.006997,0.059463,0.216320,0.059055,0.006859,0.001269,0.007497,0.059055,0.089229,0.024022,0.000769,0.001584,0.006859,0.024022,0.107099,0
3,3,0.315607,0.008904,0.003345,0.007128,0.007359,0.002430,0.006175,0.122578,0.008904,0.003430,0.000138,0.002015,0.000108,0.000384,0.000038,0.006698,0.003345,0.000138,0.000677,0.000261,0.000131,0.000085,0.000292,0.002276,0.007128,0.002015,0.000261,0.003583,0.000046,0.000161,0.000038,0.008551,0.007359,0.000108,0.000131,0.000046,0.001784,0.000369,0.001546,...,0.053619,0.416569,0.055426,0.004560,0.001238,0.004591,0.055426,0.085215,0.017947,0.001207,0.001138,0.004560,0.017947,0.047598,0.053334,0.018670,0.004075,0.000654,0.000169,0.018670,0.085922,0.052511,0.005037,0.000577,0.004075,0.052511,0.412940,0.049197,0.003699,0.000654,0.005037,0.049197,0.082646,0.016894,0.000169,0.000577,0.003699,0.016894,0.062192,0
4,4,0.306625,0.014918,0.007543,0.006659,0.010788,0.004990,0.009212,0.124054,0.014918,0.007843,0.000200,0.001707,0.000331,0.002222,0.000115,0.008635,0.007543,0.000200,0.001907,0.000454,0.000631,0.000115,0.001392,0.003929,0.006659,0.001707,0.000454,0.001984,0.000123,0.000638,0.000154,0.007643,0.010788,0.000331,0.000631,0.000123,0.003445,0.000400,0.001707,...,0.077887,0.352685,0.069344,0.006467,0.000607,0.005775,0.069344,0.088121,0.014641,0.000208,0.000754,0.006467,0.014641,0.045691,0.052258,0.019731,0.003929,0.000454,0.000046,0.019731,0.104208,0.070882,0.006090,0.000508,0.003929,0.070882,0.274406,0.073688,0.005321,0.000454,0.006090,0.073688,0.115803,0.020293,0.000046,0.000508,0.005321,0.020293,0.051443,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

## Detection Strategies

In [0]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Sample-aware detection and model-aware detection

In [7]:

concatDf = pd.concat([df,df2])
randomDf = concatDf.sample(frac = 1)
y = randomDf.pop('y')
randomDf.pop('Unnamed: 0')
print(randomDf)
X_train, X_test,y_train,y_test = train_test_split(randomDf,y, test_size=0.1,)
  


            0         1         2  ...       161       162       163
66   0.324296  0.013849  0.006005  ...  0.004598  0.023276  0.054149
374  0.352454  0.011596  0.010934  ...  0.004298  0.014841  0.047044
461  0.304764  0.020070  0.012172  ...  0.007090  0.022899  0.081954
370  0.352716  0.013295  0.006859  ...  0.005867  0.014310  0.021069
153  0.320066  0.011527  0.012780  ...  0.007051  0.020708  0.095026
..        ...       ...       ...  ...       ...       ...       ...
174  0.321220  0.009343  0.004598  ...  0.005667  0.022076  0.063653
199  0.341520  0.012465  0.007543  ...  0.005121  0.017301  0.060039
8    0.297136  0.020731  0.013972  ...  0.008974  0.029520  0.123816
424  0.359267  0.015117  0.008989  ...  0.004775  0.020600  0.060470
86   0.343935  0.009727  0.005929  ...  0.005752  0.018962  0.052642

[1000 rows x 164 columns]


Training binary SVM classifier


In [8]:
clf = svm.SVC()
clf.fit(X_train,y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
res = clf.predict(X_test)


In [10]:
print(classification_report(res,y_test,target_names=['real','fake']))

              precision    recall  f1-score   support

        real       0.72      0.88      0.79        50
        fake       0.85      0.66      0.74        50

    accuracy                           0.77       100
   macro avg       0.78      0.77      0.77       100
weighted avg       0.78      0.77      0.77       100



# Sample-aware detection
Random image is taken from a set of fake images that were generated by known generator. 
Predict : 1 - Fake Image and 0 - Real Image


In [11]:
testImg = cv2.imread('/content/drive/My Drive/Colab Notebooks/dataset/995.jpg') #this is fake image
feature1 = feature_extractor(testImg)
feature1 = feature1.reshape(-1,1)
# print(feature1.shape)
feature1 = feature1.T

dftest = pd.DataFrame(feature1)
# print(dftest)
res1 = clf.predict(dftest)
print(res1)

testImg = cv2.imread('/content/drive/My Drive/Colab Notebooks/dataset/cropped/img_align_celeba/004040.jpg') 
#above is real image
feature1 = feature_extractor(testImg)
feature1 = feature1.reshape(-1,1)
# print(feature1.shape)
feature1 = feature1.T

dftest = pd.DataFrame(feature1)
# print(dftest)
res1 = clf.predict(dftest)
print(res1)

[1]
[0]


# Model-aware detection

In [0]:
real = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresReal_amarCo.csv')
fake = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresFake_amarCo.csv')
real.pop('Unnamed: 0')
fake.pop('Unnamed: 0')
real1 = real.iloc[:450,:]
real2 = real.iloc[450:,:]
fake1 = fake.iloc[:450,:]
fake2 = fake.iloc[450:,:]


In [0]:
concatDf = pd.concat([real1,fake1])
randomDf = concatDf.sample(frac = 1)

y = randomDf.pop('y')
X_train, X_test,y_train,y_test = train_test_split(randomDf,y, test_size=0.1,)

In [57]:
clf = svm.SVC()
clf.fit(X_train,y_train)
res = clf.predict(X_test)
print(classification_report(res,y_test,target_names=['real','fake']))

              precision    recall  f1-score   support

        real       0.89      0.95      0.92        42
        fake       0.96      0.90      0.92        48

    accuracy                           0.92        90
   macro avg       0.92      0.92      0.92        90
weighted avg       0.92      0.92      0.92        90



Detecting fake image that were generated by known generative model but not used while traing classifier.

In [58]:
real1 = real.iloc[:450,:]
real2 = real.iloc[450:,:]
fake1 = fake.iloc[:450,:]
fake2 = fake.iloc[450:,:]
y_test = fake2.pop('y')
res_test = clf.predict(fake2)
print("Only fake images in fake2 for model-aware")
print(classification_report(res_test,y_test,target_names=['real','fake']))
print('no real image present so precision is 0')

Only fake images in fake2 for model-aware
              precision    recall  f1-score   support

        real       0.00      0.00      0.00         4
        fake       0.92      1.00      0.96        46

    accuracy                           0.92        50
   macro avg       0.46      0.50      0.48        50
weighted avg       0.85      0.92      0.88        50

no real image present so precision is 0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model-unaware detection

In [0]:
real = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresReal_amarCo.csv')
fake = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/featuresFake_amarCo.csv')

fake1 = fake.iloc[299,:]

real.pop('Unnamed: 0')
fake1.pop('Unnamed: 0')

y_fake1 = fake1.pop('y')

real1 = real.iloc[:490,:]
real2 = real.iloc[490:,:]
y_real1 = real1.pop('y')
y_real2 = real2.pop('y')

fake1 = pd.DataFrame(fake1)
fake1 = fake1.T



Trainig one-class classifier

In [50]:
# print(pd.DataFrame(fake1))

clf = svm.OneClassSVM(nu=0.1, kernel="rbf",gamma = 'auto')
clf.fit(real1)


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
            max_iter=-1, nu=0.1, shrinking=True, tol=0.001, verbose=False)

predicting a fake image

In [52]:
y_pred_fake = clf.predict(fake1)
print(y_pred_fake)

[1]
